### LLAMA API的请求参数

```python
api_request_json = {
    # 这是一条正常的聊天prompt
    "messages": [
        {"role": "user", "content": "What is the weather like in Boston?"},
    ],

    # 可以调用的function(s)
    "functions": [
        {   
            # function的名字
            "name": "get_current_weather",

            # function的documentation
            "description": "Get the current weather in a given location",

            # function的参数列表
            "parameters": {
                # 写死object
                "type": "object",

                # 里面罗列参数
                "properties": {

                    # 参数: location
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },

                    # 参数: days
                    "days": {
                        "type": "number",
                        "description": "for how many days ahead you wants the forecast",
                    },

                    # 参数: unit
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
            },

            # 必须参数列表
            "required": ["location", "days"],
        }
    ],
    "stream": False,

    # 这个参数表明，当前runtime下，你可以通过名字找到这个function
    # 这个function必须在当前的作用域下，可以通过名字找到
    # 这样llamaapi就会直接和这个function交互
    # 这样其实就不好了，因为你决定直接让它来调用这个函数，属实没有必要
    "function_call": "get_current_weather",
}
```

## Design Functions for OpenAI and Llama

In [25]:
from __future__ import annotations
import inspect
from pydantic import BaseModel
from typing import Dict, Optional
import json


"""
openai function calling API
https://platform.openai.com/docs/guides/function-calling

openai function requires a tool parameters, which is in fact a tool specification.
the format of the specification is defined in the link above.

The following code is a schema of the tool specification.
"""

def get_function_arg_type(func) -> dict:
    
    """Convert python function arguemnt schema to json schema, as required by openapi"""
    JSON_SCHEMA_TYPE_MAP = {
        dict:  "object",
        list:  "array",
        bool:  "boolean",
        int:   "number",
        float: "number",
        str:   "string"
    }
    
    JSON_SCHEMA_TYPE_MAP_STR = {
        "dict":"object",
        "list":"array",
        "bool":"boolean",
        "int":"number",
        "float":"number",
        "str":"string"
    }

    parsed_args = {name:type.annotation 
                   for name, type  in inspect.signature(func).parameters.items()}

    json_types = {
        arg: JSON_SCHEMA_TYPE_MAP[arg_type] 
        if arg_type in JSON_SCHEMA_TYPE_MAP.keys() 
        else JSON_SCHEMA_TYPE_MAP_STR[arg_type]
    for arg, arg_type in parsed_args.items()
    }

    return json_types

class FuncArg(BaseModel):
    """
    Python function argument wrapper.

    Attributes:
        type (Optional[str]): The type of the argument.
        description (Optional[str]): The description of the argument.
        enum (Optional[list]): The list of possible values for the argument.
    """
    type: Optional[str] = None
    description: Optional[str]
    enum: Optional[list]

    class Config:
        extra = "allow"
    

class Parameters(BaseModel):
    """
    Represents the parameters of a tool.

    Attributes:
        type (str): The type of the parameters.
        required (Optional[list]): The list of required parameters.
        properties (Dict[str, FuncArg]): The properties of the parameters.

    Example:
        parameters = Parameters(
            type="example",
            required=["param1", "param2"],
            properties={
                "param1": FuncArg(type=str),
                "param2": FuncArg(type=int)
            }
        )
    """
    type: str
    required: Optional[list]
    properties: Dict[str, FuncArg]

class Function(BaseModel):
    """
    Represents a function.

    Attributes:
        description (Optional[str]): The description of the function.
        name (str): The name of the function.
        parameters (Parameters): The parameters of the function.
    """
    description: Optional[str]
    name: str
    parameters: Parameters

class ToolSpec(BaseModel):
    type:str = None
    function:Function 
    

class FunctionWrapper:

    def __init__(self, func, **kwargs:FuncArg):
        self.func = func
        self.funcargs = kwargs
        
    @property
    def name(self):
        return self.func.__name__
        
    @property
    def function_name(self):
        return self.func.__name__
    
    @property
    def function_description(self):
        return self.func.__doc__
    
    @property
    def required(self):
        return [
            name for name, p in inspect.signature(self.func).parameters.items() 
            if p.default is inspect._empty]
    
    @property
    def tool_spec(self) -> dict:
        raise NotImplementedError("tool_spec function not implemented.")
        
    def __call__(self, *args, **kwargs):
        return self.func(*args, **kwargs)
    
    def call(self, *args, **kwargs):
        result = self.func(*args, **kwargs)
        if not isinstance(result, str):
            result = json.dumps(result)
        return result


class OpenAITool(FunctionWrapper):
    
    def __init__(self, func, **kwargs:FuncArg):
        super().__init__(func, **kwargs)
        
    @property
    def tool_spec(self) -> dict:
        arg_types = get_function_arg_type(self.func)
        
        funcargs = {}
        for k,v in self.funcargs.items():
            # check if input type is not specified
            # and can be retrieved from signature
            if v.type is None and k in arg_types:
                v.type = arg_types[k]
                funcargs[k] = v
            else:
                funcargs[k] = v      
            
        parameters = Parameters(
                    type='object',
                    properties=funcargs)
        
        if len(self.required) > 0:
            parameters.required = self.required
        
        return ToolSpec(
            type="function",
            function=Function(
                name=self.function_name,
                description=self.function_description,
                parameters=parameters
            )
        ).dict(exclude_unset=True)
    
class LlamaFunction(FunctionWrapper):
    
    def __init__(self, func, **kwargs:FuncArg):
        super().__init__(func, **kwargs)
        
    @property
    def tool_spec(self) -> dict:
        arg_types = get_function_arg_type(self.func)
        
        funcargs = {}
        for k,v in self.funcargs.items():
            # check if input type is not specified
            # and can be retrieved from signature
            if v.type is None and k in arg_types:
                v.type = arg_types[k]
                funcargs[k] = v
            else:
                funcargs[k] = v      
            
        parameters = Parameters(
                    type='object',
                    properties=funcargs)
        
        if len(self.required) > 0:
            parameters.required = self.required

        return Function(
                name=self.function_name,
                description=self.function_description,
                parameters=parameters
            ).dict(exclude_unset=True)

In [26]:
import random
def get_weather(location:str, unit:str, days:int):
    """Get weather forecast for a location in unit"""
    temperature = random.randint(1,100)
    return f"weather forecast for {location} in {unit} for {days} days is {temperature}"

In [35]:
llama_func = LlamaFunction(get_weather, 
              location=FuncArg(description="location of the weather"),
              unit=FuncArg(description="unit of the weather, C for Celcius, F for Farehait", enum = ['C','F']),
              days=FuncArg(description="number of days to forecast"))

openai_tool = OpenAITool(get_weather, location = FuncArg(description="location of the weather"),
                  unit = FuncArg(description="unit of the weather, C for Celcius, F for Farehait", enum = ['C','F']),
                  days = FuncArg(description="number of days to forecast"))